In [2]:
# from comet_ml import Experiment
# from comet_ml.api import API, APIExperiment
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level, save
import pickle
import os

# experiment = Experiment(
#     api_key=os.environ.get('COMET_ML_API_KEY'),
#     project_name="inflow",
#     workspace="drdevinhopkins",
# )
# api = API(os.environ.get('COMET_ML_API_KEY'))

set_log_level("ERROR")

target_column = 'Total Inflow hrly'


In [3]:

# df = pd.read_csv(
#     'https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/since-2020.csv')
df = pd.read_csv(
    '../data/since-2020.csv')
df.ds = pd.to_datetime(df.ds)
df = df.drop(['Date', 'Time'], axis=1)
df = df.sort_values(by='ds', ascending=True)
df.rename(columns={target_column: 'y', 'Adm. requests cum': "Adm requests cum",
          'Pts.waiting for admission CUM': 'Pts waiting for admission CUM'}, inplace=True)
df.dropna(inplace=True)
regressors = df.columns.tolist()
regressors.remove('y')
regressors.remove('ds')

df


,Stretcher Pts hrly,Stretcher Pts cum,Ambulatory Pts hrly,Ambulatory Pts cum,y,Total Inflow cum,Ambulances hrly,Ambulances cum,FLS hrly,Adm requests cum,...,Consults > 2h in Vertical Except IM,Consult for IM >4h in Vertical,Plain films reqs > 2 hr in Vertical,CTs reqs > 2 hrs in Vertical,Psych Stretcher Pts1pt,Psych pts waiting for admission,ds,Total Pod TBS,Total Vertical TBS,Stretcher Overflow
0,3,3,1,1,4,4,3,3,0,0,...,2,0,0,1,6,4,2020-01-01 01:00:00,2,5,1
1,5,8,1,2,6,10,2,5,1,1,...,3,0,0,1,7,4,2020-01-01 02:00:00,3,7,0
2,7,15,2,4,9,19,2,7,1,1,...,1,0,0,1,7,4,2020-01-01 03:00:00,0,11,0
3,6,21,0,4,6,25,5,12,0,2,...,2,0,0,1,7,4,2020-01-01 04:00:00,3,8,0
4,2,23,1,5,3,28,0,12,0,3,...,2,0,0,0,9,4,2020-01-01 05:00:00,2,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29812,3,10,1,9,4,19,2,6,0,3,...,2,2,0,4,17,15,2023-06-08 06:00:00,2,3,0
29813,2,12,1,10,3,22,1,7,1,3,...,2,2,0,4,19,15,2023-06-08 07:00:00,1,3,0
29814,8,20,3,13,11,33,1,8,0,4,...,2,1,1,4,19,15,2023-06-08 08:00:00,2,11,0
29815,8,28,7,20,15,48,1,9,0,5,...,2,1,1,4,19,15,2023-06-08 09:00:00,2,11,0


In [5]:

params = {
    # growth='off',
    'yearly_seasonality': True,
    'weekly_seasonality': True,
    'daily_seasonality': True,
    'n_lags': 4,
    'n_forecasts': 12,
    'changepoints_range': 0.95,
    'n_changepoints': 50,
    'quantiles': [0.2, 0.5, 0.8]
    # num_hidden_layers=4,
    # d_hidden=36,
    # learning_rate=0.005,
}

m = NeuralProphet(**params)

m = m.add_lagged_regressor(names=regressors)
m = m.add_country_holidays("CA")


In [6]:

metrics = m.fit(df,
                freq='H',
                # progress='plot'
                )
print(metrics.tail(1))


: 

: 

In [ ]:

# experiment.log_parameters(params)
# experiment.log_metrics(metrics.tail(1).iloc[0].to_dict())

save(m, "models/inflow.np")

# experiment.log_model("inflow", "models/inflow.np")

# with open('models/inflow_forecast_model.pkl', "wb") as f:
#     pickle.dump(m, f)

# experiment_name = experiment.name

# experiment.end()

# try:
#     api.delete_registry_model('drdevinhopkins', 'inflow')
# except:
#     print('Failed to delete previous model')

# apiexp = api.get('drdevinhopkins/inflow/'+experiment_name)

# apiexp.register_model('inflow')